### Prerequisites to run this notebook

## Track the datasets in your Azure ML experiment using Python SDKv2 and MLflow ##

There are two main approaches to track the datasets: the **first** one is to save a copy of your input data:


Prerequisites to run this notebook:   
insatll Python SDKv2: InstallSDKv2.ipynb

In [2]:
# Ensure you have the dependencies for this notebook
#%pip install -r logging_model_with_mlflow.txt

In [3]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace
from mlflow.tracking import MlflowClient

import pandas as pd

from azureml.core import Dataset
#from azureml.data.dataset_factory import DataType
import tempfile

print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)

SDK version: 1.44.0
MLflow version: 1.28.0


In [4]:
experiment_name ="TrackDatasets"
experiment=mlflow.set_experiment(experiment_name)
experiment_id=experiment.experiment_id

# Create an experiment with a name that is unique and case sensitive.
client = MlflowClient()
#experiment_id = client.create_experiment(experiment_name)
client.set_experiment_tag(experiment_id, "exper ver", "1")


# Fetch experiment metadata information
experiment = client.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))

Name: TrackDatasets
Experiment_id: 1ae4018f-adf2-4b54-ae2c-ad04fe6ddca0
Artifact Location: 
Tags: {'exper ver': '1'}
Lifecycle_stage: active


In [4]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
df["thal"] = df["thal"].astype("category").cat.codes

In [29]:
#Generate Hash from the dataframe 
import joblib
HFile=joblib.hash(df)
print(HFile)

0c42457f54eff24857ffb448e9ece515


In [32]:
import os
import time

path = r"./saveDatasetVer/train_dataset.csv"

ti_m = os.path.getmtime(path)
 
m_ti = time.ctime(ti_m)
 
# Using the timestamp string to create a
# time object/structure
t_obj = time.strptime(m_ti)
 
# Transforming the time object to a timestamp
# of ISO 8601 format
TStamp = time.strftime("%Y-%m-%d %H:%M:%S", t_obj)
 
FName=os.path.basename(path)

print(f"The file located at the path {path} was last modified at {T_stamp}")

The file located at the path ./saveDatasetVer/train_dataset.csv was last modified at 2022-10-04 11:03:21


In [31]:
FName, TStamp, HFile

('train_dataset.csv',
 '2022-10-04 11:03:21',
 '0c42457f54eff24857ffb448e9ece515')

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

There are two main approaches to track the datasets: the first one is to save a copy of your input data:

```
    with tempfile.TemporaryDirectory() as tmp:
        path = 'saveDatasetVer/train_dataset.csv'  #path where you whant to save your dataset used for training 
        X_train.to_csv(path)
        mlflow.log_artifacts(tmp)
```

In case the dataset is too big you can log the path to the dataset as a parameter: 

```
    mlflow.log_param('dsPathМ', file_url )   #track the pointer to the data
    mlflow.log_param('FileName', FName)  #track the source FileName
    mlflow.log_param('LastModified', TStamp) #track the Last Modified date of the source file
    mlflow.log_param('HashedData', HFile) #track the  data (hashed data frame)

```


In [35]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature

with mlflow.start_run():  
    mlflow.xgboost.autolog(log_models=True,log_input_examples=True,log_model_signatures=True,registered_model_name='DatastetsModel')

    model = XGBClassifier(use_label_encoder=False, eval_metric="auc")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    mlflow.log_param('dsPathМ', file_url )   #track the pointer to the data
    mlflow.log_param('FileName', FName)   #track the source FileName
    mlflow.log_param('LastModified', TStamp) #track the Last Modified date of the source file
    mlflow.log_param('HashedData', HFile) #track the  data (hashed data frame)

    with tempfile.TemporaryDirectory() as tmp:
        path = 'saveDatasetVer/train_dataset.csv'  #path where you whant to save your dataset used for training 
        X_train.to_csv(path)
        mlflow.log_artifacts(tmp)

    signature = infer_signature(X_test, y_test)
    mlflow.xgboost.log_model(model, "classifier", signature=signature)

2022/10/05 11:31:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Registered model 'DatastetsModel' already exists. Creating a new version of this model...
2022/10/05 11:31:37 INFO mlflow.tracking._model_re

In [36]:
mlflow.end_run()

**log_models** -  If True, trained models are logged as MLflow model artifacts. If False, trained models are not logged. Input examples and model signatures, which are attributes of MLflow models, are also omitted when log_models is False.  


**registered_model_name** – If given, each time a model is trained, it is registered as a new model version of the registered model with this name. The registered model is created if it does not already exist.    

**log_input_examples** – If True, input examples from training datasets are collected and logged along with XGBoost model artifacts during training. If False, input examples are not logged. Note: Input examples are MLflow model attributes and are only collected if log_models is also True

**log_model_signatures** – If True, ModelSignatures describing model inputs and outputs are collected and logged along with XGBoost model artifacts during training. If False, signatures are not logged. Note: Model signatures are MLflow model attributes and are only collected if log_models is also True.
